Acquire

In [193]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats
import xgboost as xgb

from sklearn.model_selection import train_test_split

from pandas.core.window.ewm import ExponentialMovingWindow as emw

import wrangle
from wrangle import collapse_columns, get_null_count, get_zeros, get_delta_values, get_ema, get_pctb, get_range, get_cv

import warnings
warnings.filterwarnings("ignore")

Here we analyzed the feature importance based on the Gain metric from the best performing XGBoost model at this point. 
The goal is to identify the raw features that are providing the most information to the model as a way to reduce the number of features and thus the dimensionality. In doing that, we can focus on generating new features from the most influential and decrease performance time. We will take the top 20 features for a first run. 

In [195]:
df = pd.read_csv('model8_feature_scores.csv')
df['raw_feature'] = df['0'].replace(to_replace = '_[a-z]+$', value = '', regex=True)
cols_to_keep = df.groupby('raw_feature').sum().sort_values(by='gain').tail(100).index

In [196]:
# %%timeit -r 1 -n 1
# X_df_header = pd.read_csv('../../data/raw/train_data.csv', nrows=0)
X_df, y_df = wrangle.acquire_amex()

In [197]:
# create dataframe of categorical columns only
# cat_columns = ['B_30', 'B_31', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
# X_df_cat = X_df[['customer_ID'] + cat_columns]

# # we will want to create dummy variables of categorical columns
# X_df_cat = pd.get_dummies(X_df_cat, columns=cat_columns, drop_first=True)

Create new features out of numeric variables

Are there any categorical variables in the top 20? 

In [198]:
cols_to_keep

Index(['D_130', 'B_24', 'D_145', 'R_12', 'D_111', 'D_137', 'S_16', 'D_118',
       'R_7', 'B_33', 'D_61', 'S_6', 'D_132', 'D_128', 'D_102', 'D_133',
       'B_29', 'D_144', 'D_58', 'D_112', 'D_59', 'S_7', 'D_74', 'R_16', 'P_3',
       'D_52', 'B_36', 'D_80', 'B_15', 'B_26', 'R_5', 'D_55', 'S_5', 'B_6',
       'S_12', 'S_9', 'B_21', 'S_20', 'D_60', 'D_106', 'B_32', 'D_129',
       'D_121', 'R_18', 'D_56', 'B_5', 'D_140', 'R_4', 'R_8', 'S_11', 'D_77',
       'B_14', 'R_21', 'S_24', 'B_40', 'D_79', 'D_120', 'S_26', 'D_46', 'D_42',
       'S_8', 'D_54', 'D_70', 'R_11', 'S_15', 'R_26', 'B_10', 'B_11', 'S_25',
       'D_47', 'R_2', 'D_50', 'D_48', 'B_8', 'R_27', 'D_39', 'R_6', 'S_23',
       'B_37', 'S_13', 'D_49', 'B_23', 'B_2', 'D_62', 'B_1', 'D_51', 'D_65',
       'B_4', 'B_3', 'D_41', 'R_3', 'D_45', 'S_3', 'D_44', 'B_7', 'R_1',
       'B_18', 'D_75', 'P_2', 'B_9'],
      dtype='object', name='raw_feature')

Nope, so I will create a new dataframe with the top 20 features and then work on adding the new features. 

In [199]:
# num_columns = [col for col in X_df.columns if col[0] not in cat_columns]
# X_df_num = X_df[num_columns]

In [200]:
X_df20 = X_df[['customer_ID'] + list(cols_to_keep)]

In [201]:
missing_vals_df = get_null_count(X_df20[['customer_ID'] + list(X_df20.isnull().sum()[X_df20.isnull().sum()>0].index)])
missing_vals_df

,D_130_nulls,D_145_nulls,R_12_nulls,D_111_nulls,D_137_nulls,D_118_nulls,B_33_nulls,D_61_nulls,D_132_nulls,D_128_nulls,...,B_37_nulls,D_49_nulls,B_2_nulls,D_62_nulls,B_3_nulls,D_41_nulls,D_45_nulls,S_3_nulls,D_44_nulls,P_2_nulls
customer_ID,,,,,,,,,,,,,,,,,,,,,
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.0,0.0,0.0,13.0,13.0,0.0,0.0,2.0,13.0,0.0,...,0.0,13.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5,0.0,0.0,0.0,13.0,13.0,0.0,0.0,0.0,13.0,0.0,...,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1,0.0,0.0,0.0,13.0,13.0,0.0,0.0,0.0,13.0,0.0,...,0.0,13.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0
000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc,0.0,0.0,0.0,13.0,13.0,0.0,0.0,4.0,13.0,0.0,...,0.0,13.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed,0.0,0.0,0.0,13.0,13.0,0.0,0.0,0.0,13.0,0.0,...,0.0,13.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0956e6df4034f0e007c8c3a57aa260c1d10a6cb506462bf5b810ef45c471a7ad,0.0,0.0,0.0,9.0,9.0,2.0,0.0,0.0,9.0,0.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0956f2eafc5d8919835f76f6e40835b49900cf91c3de18c3c80da1ae33fb06eb,1.0,1.0,0.0,13.0,13.0,0.0,0.0,11.0,13.0,1.0,...,0.0,13.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0
0956f6dbb2671e10a40180b39ea793fb73633767ef70be12f3946a08b46b6c99,0.0,0.0,0.0,13.0,13.0,0.0,0.0,0.0,13.0,0.0,...,0.0,13.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0


In [202]:
# fill all missing values with -1
X_df20 = X_df20.fillna(value=-1)

In [203]:
# create dataframe with the last value, standard deviation, min and max of each variable for each customer. 
agg_df = X_df20.groupby('customer_ID').agg(['last', 'std', 'min', 'max'])
agg_df = collapse_columns(agg_df)

In [204]:
# create dataframe with the number of records of 0 value of each variable for each customer. 
zero_df = get_zeros(X_df20)

In [205]:
# create dataframe with the difference between the last value and the value 2 months prior for each variable for each customer. 
delta_df = get_delta_values(X_df20)

In [206]:
# create dataframe with the exponential moving average, with a weight of .8, for each variable for each customer. 
ema_df = get_ema(X_df20)

In [207]:
# aggregate all of the above dataframes into a single 'metrics' dataframe
metrics_df = pd.concat([agg_df, missing_vals_df, zero_df, delta_df, ema_df],axis=1)

In [208]:
# create a new feature, pctb, which is the percent b value for each variable for each customer. 
metrics_df = get_pctb(X_df20, metrics_df)

In [209]:
# create a new feature, range, which is the max - min for each variable for each customer. 
metrics_df = get_range(X_df20, metrics_df)

In [210]:
# create a new feature, cv, which represents the coefficient of variation (std/mean) for each variable for each customer. 
metrics_df = get_cv(X_df20, metrics_df)

In [211]:
# drop the _min and _std columns. The info for these is captured in _range and _cv
cols_to_drop = metrics_df.filter(regex='(_min|_std)$', axis=1).columns
metrics_df = metrics_df.drop(columns=cols_to_drop)

In [222]:
for col in cols_to_keep:
    metrics_df.loc[metrics_df[col+'_ema'].isnull(), col+'_ema'] = metrics_df.loc[:,col+'_last']
    metrics_df.loc[metrics_df[col+'_pctb'].isnull(), col+'_pctb'] = .5
    metrics_df[col+'_cv'].fillna(value=0, inplace=True)
    metrics_df[col+'_diff'].fillna(value=0, inplace=True)
    metrics_df[col+'_diff_mean'].fillna(value=0, inplace=True)

In [223]:
# drop those columns where > 90% of rows are missing values
missing_counts_df = pd.DataFrame({'missing_count': metrics_df.isnull().sum(), 'missing_pct': metrics_df.isnull().sum()/len(metrics_df)})

In [224]:
cols_to_drop = missing_counts_df[missing_counts_df.missing_pct > .90].index
features_df = metrics_df.drop(columns=cols_to_drop)

Missing values that are returned after the creation of new features are due to the following:

1. _diff, _ema, _cv, _%b when the customer only has one month of data. 
2. _diff_mean when the customer only has two months of data. 

I will replace missing values with the following: 

1. Fill cv with 0
2. Fill %b with .5 because that is the value when the last value is equal to the mean. 
3. Fill diff with 0
4. Fill diff_mean with 0
5. Fill ema with last value

In [225]:
features_df

,D_130_cv,B_24_cv,D_145_cv,R_12_cv,D_111_cv,D_137_cv,S_16_cv,D_118_cv,R_7_cv,B_33_cv,...,R_3_ema,D_45_ema,S_3_ema,D_44_ema,B_7_ema,R_1_ema,B_18_ema,D_75_ema,P_2_ema,B_9_ema
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.362792,0.362901,0.702649,0.003386,-0.0,-0.0,0.609410,0.030623,0.496798,0.002638,...,0.009471,0.738224,0.102254,0.004734,0.050152,0.001864,1.005599,0.009480,0.930801,0.002200
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5,2.577358,0.414642,0.781377,0.002618,-0.0,-0.0,4.340647,0.016420,0.349898,0.003218,...,0.107610,0.264172,0.153820,0.000704,0.030948,0.007023,1.001328,0.009064,0.878641,0.011141
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1,0.424107,1.032012,0.446131,0.002569,-0.0,-0.0,1.128064,0.019532,0.582259,0.002839,...,0.007667,0.245188,-1.000000,0.004750,0.041081,0.006869,1.007355,0.007581,0.871441,0.001297
000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc,1.473677,0.507735,0.598471,0.002784,-0.0,-0.0,0.724666,0.123998,1.204975,0.003765,...,0.003314,0.075031,0.315203,0.001619,0.016453,0.004706,0.916407,0.017184,0.621672,0.039338
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed,0.002555,0.725755,0.975665,0.003001,-0.0,-0.0,0.964257,0.017912,1.004179,0.002743,...,0.006401,0.070566,-0.998434,0.006451,0.165520,0.005969,0.530599,0.136172,0.879369,0.009378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0956e6df4034f0e007c8c3a57aa260c1d10a6cb506462bf5b810ef45c471a7ad,0.306158,0.706621,0.442439,0.003560,-0.0,-0.0,1.272244,4.110691,0.852128,62.325635,...,0.029796,0.024672,0.272118,0.128987,0.261113,0.006707,0.183599,0.271833,0.494758,0.405769
0956f2eafc5d8919835f76f6e40835b49900cf91c3de18c3c80da1ae33fb06eb,372.660223,2.522388,72.351548,0.002966,-0.0,-0.0,0.472626,0.358871,0.339495,0.372159,...,0.009018,0.485926,0.146060,0.002372,0.024632,0.408927,1.002545,0.001626,0.824353,0.002735
0956f6dbb2671e10a40180b39ea793fb73633767ef70be12f3946a08b46b6c99,0.002606,0.353969,0.304201,0.002626,-0.0,-0.0,0.709301,0.024026,0.472761,0.002463,...,0.608237,0.286118,0.173424,0.001656,0.141168,0.006024,0.591200,0.137344,0.878155,0.002146
095700a5c40952f9c134c6a71700bbd959098adfdcf2c7330351d9468f8b33a7,111.501245,0.379405,68.994665,0.003342,-0.0,-0.0,0.371714,0.009092,0.636491,0.441079,...,0.008576,0.410618,0.054797,0.231143,0.244303,0.005611,0.477315,0.409042,0.692135,0.008839


In [226]:
# missing values complete!
features_df.isnull().sum().sum()

0

In [227]:
features_df = features_df.reset_index()

In [228]:
features_df = features_df.rename(columns={'index': 'customer_ID'})

In [229]:
X_train, y_train, X_validate, y_validate, X_test, y_test = wrangle.split_amex(features_df, 
                                                                              y_df, 
                                                                              train_size=.5, 
                                                                              test_size=.5)

In [230]:
train = X_train.merge(y_train, how='left', on='customer_ID')
validate = X_validate.merge(y_validate, how='left', on='customer_ID')
test = X_test.merge(y_test, how='left', on='customer_ID')

In [244]:
for col in train.columns:
    print(col)
    print(train[col].describe()[-1])

customer_ID
1
D_130_cv
7719.814253587479
B_24_cv
754.0446897697166
D_145_cv
12330.781757683808
R_12_cv
177.73019940639088
D_111_cv
1.4879383757233888
D_137_cv
347782.9774024108
S_16_cv
1152.2679148361458
D_118_cv
1825.5512636709955
R_7_cv
3072.3390960814427
B_33_cv
722.4339945577558
D_61_cv
3604.946062678319
S_6_cv
870.7145095349755
D_132_cv
362.4653029740862
D_128_cv
1180.6574071890614
D_102_cv
99648.5867698253
D_133_cv
115980.42913147694
B_29_cv
537.3282914071152
D_144_cv
2744.8247251797598
D_58_cv
935.5550170951987
D_112_cv
346.1198258301546
D_59_cv
76.738911619265
S_7_cv
274.45011337117836
D_74_cv
79.94071785299614
R_16_cv
953.9272825796779
P_3_cv
2759.0142553759783
D_52_cv
420.93156730650884
B_36_cv
19.088621555828194
D_80_cv
450.6936162123228
B_15_cv
191.47886243159124
B_26_cv
5502.872225192071
R_5_cv
588.8708420015147
D_55_cv
163.74699201748558
S_5_cv
407.60488549425463
B_6_cv
577.2191995245594
S_12_cv
3809.352796107281
S_9_cv
875.5640543049865
B_21_cv
959.542853184259
S_20_cv
3

1.0098682476161849
D_52_max
1.009960697602453
B_36_last
0.4409783359964303
B_36_max
0.5232614501532646
D_80_last
3.407161161366594
D_80_max
3.803990664903788
B_15_last
7.592668780393219
B_15_max
9.344417137232131
B_26_last
163.30374576079487
B_26_max
196.9627069378628
R_5_last
9.503413260741867
R_5_max
9.508355687394694
D_55_last
1.1638535192921646
D_55_max
1.1638535192921646
S_5_last
36.412302005413885
S_5_max
36.48781272085778
B_6_last
8.201710521404962
B_6_max
16.4889695734372
S_12_last
16.746990263649632
S_12_max
16.746990263649632
S_9_last
2.5711449557711896
S_9_max
2.835733963318673
B_21_last
27.009886361877275
B_21_max
27.009886361877275
S_20_last
1.0099599865256232
S_20_max
1.0099893199783176
D_60_last
1.0099586754314822
D_60_max
1.009999655088312
D_106_last
2.3559172045963903
D_106_max
15.965540720921435
B_32_last
1.0099186500787487
B_32_max
1.0099936071080942
D_129_last
1.0099992806826608
D_129_max
1.0099999888347952
D_121_last
1.3236950944211208
D_121_max
1.325574113894485
R

0.5978417160100593
P_2_diff
1.6901847278533249
B_9_diff
0.9919020631780215
D_130_diff_count
9.0
B_24_diff_count
12.0
D_145_diff_count
9.0
R_12_diff_count
10.0
D_111_diff_count
9.0
D_137_diff_count
9.0
S_16_diff_count
10.0
D_118_diff_count
8.0
R_7_diff_count
10.0
B_33_diff_count
10.0
D_61_diff_count
12.0
S_6_diff_count
10.0
D_132_diff_count
3.0
D_128_diff_count
9.0
D_102_diff_count
9.0
D_133_diff_count
10.0
B_29_diff_count
9.0
D_144_diff_count
10.0
D_58_diff_count
12.0
D_112_diff_count
9.0
D_59_diff_count
10.0
S_7_diff_count
12.0
D_74_diff_count
11.0
R_16_diff_count
10.0
P_3_diff_count
12.0
D_52_diff_count
10.0
B_36_diff_count
12.0
D_80_diff_count
10.0
B_15_diff_count
12.0
B_26_diff_count
10.0
R_5_diff_count
10.0
D_55_diff_count
12.0
S_5_diff_count
11.0
B_6_diff_count
12.0
S_12_diff_count
10.0
S_9_diff_count
10.0
B_21_diff_count
9.0
S_20_diff_count
9.0
D_60_diff_count
12.0
D_106_diff_count
8.0
B_32_diff_count
10.0
D_129_diff_count
9.0
D_121_diff_count
8.0
R_18_diff_count
9.0
D_56_diff_c

In [245]:
cols_to_drop = ['D_120_cv']
train = train.drop(columns=cols_to_drop)
validate = validate.drop(columns=cols_to_drop)
test = test.drop(columns=cols_to_drop)
# train[np.isinf(np.array(train.D_120_cv))].filter(regex='D_120', axis=1)

In [246]:
train_matrix = xgb.DMatrix(train.drop(columns=['customer_ID', 'target']), label=train.target)
valid_matrix = xgb.DMatrix(validate.drop(columns=['customer_ID', 'target']), label=validate.target)

In [251]:
steps = 200
seed = 42

params = {
    'verbosity': 1,
    'max_depth': 3,
    'objective': 'binary:logistic',
    'eta': 0.15,
    'random_state': seed,
    'colsample_bytree': 0.8,
    'colsample_bylevel': 0.8
}

In [252]:
model = xgb.train(params, train_matrix, steps, early_stopping_rounds=3,
                  evals=[(train_matrix, 'Train'), (valid_matrix, 'Valid')])

[0]	Train-logloss:0.60371	Valid-logloss:0.60930
[1]	Train-logloss:0.53674	Valid-logloss:0.54708
[2]	Train-logloss:0.48482	Valid-logloss:0.49971
[3]	Train-logloss:0.44257	Valid-logloss:0.46183
[4]	Train-logloss:0.40854	Valid-logloss:0.43152
[5]	Train-logloss:0.38025	Valid-logloss:0.40672
[6]	Train-logloss:0.35675	Valid-logloss:0.38553
[7]	Train-logloss:0.33745	Valid-logloss:0.36909
[8]	Train-logloss:0.32085	Valid-logloss:0.35478
[9]	Train-logloss:0.30669	Valid-logloss:0.34314
[10]	Train-logloss:0.29421	Valid-logloss:0.33294
[11]	Train-logloss:0.28291	Valid-logloss:0.32434
[12]	Train-logloss:0.27382	Valid-logloss:0.31724
[13]	Train-logloss:0.26532	Valid-logloss:0.31061
[14]	Train-logloss:0.25724	Valid-logloss:0.30515
[15]	Train-logloss:0.25075	Valid-logloss:0.30097
[16]	Train-logloss:0.24488	Valid-logloss:0.29684
[17]	Train-logloss:0.23991	Valid-logloss:0.29314
[18]	Train-logloss:0.23447	Valid-logloss:0.29019
[19]	Train-logloss:0.22980	Valid-logloss:0.28743
[20]	Train-logloss:0.22578	Val

In [ ]:
#################################
# Feature Engineering functions #
#################################

def collapse_columns(X_df):
    '''
    this function will collapse the multi-level index of the columns 
    that are generated after computing the first set of aggregates in 
    our groupby function in the agg_features function.
    '''
    # df = X_df.copy()
    if isinstance(X_df.columns, pd.MultiIndex):
        X_df.columns = X_df.columns.to_series().apply(lambda x: "_".join(x))
    return X_df

def get_null_count(X_df):
    '''
    this function will calculate the number of missing values for each feature. 
    it reaturns a dataframe with the columns: <column_name_orig>_nulls 
    '''
    missing_vals = X_df.groupby('customer_ID').agg(lambda x: x.isnull().sum())
    missing_vals.columns = [x + '_nulls' for x in missing_vals.columns]
    return missing_vals

def get_zeros(X_df):
    '''
    this function will calculate the number of zeros values for each feature. 
    it reaturns a dataframe with the columns: <column_name_orig>_zeros 
    '''
    zeros_df = X_df.groupby('customer_ID').agg(lambda x: (x == 0.0).sum())
    zeros_df.columns = [x + '_zeros' for x in zeros_df.columns]
    return zeros_df

# def get_cv(X_df):
#     '''
#     this function will compute the coefficient of variation for each feature. 
#     it reaturns a dataframe with the columns: <column_name_orig>_cv 
#     '''
#     cv_df = X_df.groupby('customer_ID').agg(lambda x: x.std()/x.mean())
#     cv_df.columns = [x + '_cv' for x in cv_df.columns]
#     return cv_df

def get_two_period_difference(X_df):
    '''
    This function computes the 2-period in values for each feature. 
    it returns a dataframe with the customer id set to the index. 
    the function is used in compute_delta_values() function
    '''
    delta_df = X_df.groupby('customer_ID').diff(periods=2)
    delta_df.index = X_df.customer_ID
    return delta_df

    
def get_delta_values(X_df):
    '''
    This function first gets the two-period difference in values for each feature and assigns that to a dataframe (delta).
    It generates a dataframe of the most recent 2-period difference (delta_value).
    Next, from the delta dataframe, it computes the number of negative deltas over the customer's history and 
    assigns that to a dataframe (neg_delta_count).
    Next, it uses the delta dataframe to compute the average delta over the customer's history and assigns that to 
    a dadtaframe (delta_mean).
    Finally, all of these dataframes are concatenated into a single dataframe, delta_df. 
    '''
    # first compute the 2 period delta and create a dataframe with those values
    delta_df = get_two_period_difference(X_df)
    delta_df.columns = [x + '_diff' for x in delta_df.columns]
    
    # Use the delta df to take the last value as the current delta
    delta_value = delta_df.groupby('customer_ID').last()
    
    # use the delta df to count the number of changes over customer history that were negative
    neg_delta_count = delta_df.groupby('customer_ID').agg(lambda x: (x < 0).sum())
    neg_delta_count.columns = [x + '_count' for x in delta_df.columns]
    
    # use the delta df to compute the rolling average of the delta values
    delta_mean = delta_df.groupby('customer_ID').transform(lambda x: x.rolling(window=6, 
                                                                       min_periods=3, 
                                                                       closed='left').mean())
    delta_mean.columns = [x + '_mean' for x in delta_df.columns]
    
    # take the last value, the current average of change
    delta_mean = delta_mean.groupby('customer_ID').last()
    
    # concatenate the dataframes with the computed values by concatenating columns along the customer index
    delta_df = pd.concat([delta_value, neg_delta_count, delta_mean], axis=1)
    return delta_df

def get_ema(X_df):
    '''
    This function will compute the exponential moving average, with an alpha of .8. 
    it returns a dataframe with the columns: <column_name_orig>_ema. 
    '''
    ema_df = X_df.groupby('customer_ID').transform(lambda x: x.ewm(alpha=.8, min_periods=1, adjust=True).mean().shift(periods=1))
    ema_df.columns = [x + '_ema' for x in ema_df.columns]
    ema_df.index = X_df.customer_ID
    ema_df = ema_df.groupby('customer_ID').last()
    return ema_df

def get_pctb(X_df, metrics_df):
    df_customer_indexed = X_df.set_index('customer_ID')
    pctb_series = pd.Series()

    # loop through original column names and for eacsh one, compute pctb
    k = 6
    for x in df_customer_indexed.columns:
        ubb = metrics_df[x + '_ema'] + k*metrics_df[x + '_std']
        lbb = metrics_df[(x + '_ema')] - k*metrics_df[x + '_std']
        pctb = (metrics_df[x + '_last'] - lbb) / (ubb - lbb)
        pctb_series = pd.concat([pctb_series, pctb], axis=1)
    
    pctb_df = pd.DataFrame(pctb_series)
    pctb_df = pctb_df.iloc[:,1:]
    pctb_df.columns = [x + '_%b' for x in df_customer_indexed.columns]
    metrics_df = pd.concat([pctb_df, metrics_df], axis=1)
    return metrics_df

def get_range(X_df, metrics_df):
    range_series = pd.Series()
    for x in df_customer_indexed.columns:
        range_val = metrics_df[x + '_max'] - metrics_df[x + '_min']
        range_series = pd.concat([range_series, range_val], axis=1)

    range_df = pd.DataFrame(range_series)
    range_df = range_df.iloc[:,1:]
    range_df.columns = [x + '_%b' for x in df_customer_indexed.columns]
    metrics_df = pd.concat([range_df, metrics_df], axis=1)
    return metrics_df

def get_cv(X_df, metrics_df):
    cv_series = pd.Series()
    for x in df_customer_indexed.columns:
        cv = metrics_df[x + '_std']/metrics_df[x + '_ema']
        cv_series = pd.concat([cv_series, cv], axis=1)

    cv_df = pd.DataFrame(cv_series)
    cv_df = cv_df.iloc[:,1:]
    cv_df.columns = [x + '_cv' for x in df_customer_indexed.columns]
    metrics_df = pd.concat([cv_df, metrics_df], axis=1)
    return metrics_df

def ent(data):
    """Calculates entropy of the passed `pd.Series`
    """
    p_data = data.value_counts()           # counts occurrence of each value
    entropy = scipy.stats.entropy(p_data)  # get entropy from counts
    return entropy

def get_features(X_df):

    agg_df = X_df.groupby('customer_ID').agg(['last', 'std', 'min', 'max'])
    agg_df = collapse_columns(agg_df)

    missing_vals_df = get_null_count(X_df)
    zero_df = get_zeros(X_df)
    delta_df = get_delta_values(X_df)
    ema_df = get_ema(X_df)

    metrics_df = pd.concat([agg_df, missing_vals_df, zero_df, delta_df, ema_df],axis=1)

    metrics_df = get_pctb(X_df, metrics_df)
    metrics_df = get_range(X_df, metrics_df)
    metrics_df = get_cv(X_df, metrics_df)

    # drop the _min and _std columns. Those are captured in _range and _cv
    cols_to_drop = metrics_df.filter(regex='(_min|_std)$', axis=1).columns
    metrics_df = metrics_df.drop(columns=cols_to_drop)

    # drop those columns where > 90% of rows are missing values
    missing_counts_df = pd.DataFrame({'missing_count': metrics_df.isnull().sum(), 'missing_pct': metrics_df.isnull().sum()/len(metrics_df)})
    cols_to_drop = missing_counts_df[missing_counts_df.missing_pct > .90].index
    metrics_df = metrics_df.drop(columns=cols_to_drop)

    entropy_series = metrics_df.apply(ent)
    features_df = metrics_df[entropy_series[entropy_series > 1].index]
    return features_df



Run line by line, testing

In [ ]:
X_df

In [ ]:
features_df.to_csv('features_2.csv')

Flatten the time series data. 

For each variable, we need to create the following:



Explore the different columns, datatypes, descriptive stats

For reference: 
* D_* = Delinquency variables
* S_* = Spend variables
* P_* = Payment variables
* B_* = Balance variables
* R_* = Risk variables

In [ ]:
spend = X_df.iloc[:,X_df.columns.str[0] == 'S']
delinq = X_df.iloc[:,X_df.columns.str[0] == 'D']
pay = X_df.iloc[:,X_df.columns.str[0] == 'P']
balance = X_df.iloc[:,X_df.columns.str[0] == 'B']
risk = X_df.iloc[:,X_df.columns.str[0] == 'R']

**Spend variables**

- 22 total columns

- S_2: date *needs to be converted* **done**

- All others: float

- S_2, S_5, S_6, S_8, S_11:S_13, S_15:S_20 : no missing values

- S_22:S_26 : missing < 1% of values

- S_3, S_7, S_27 : missing 1-25% of values

- S_9, S_27 : missing 25-75% of values

In [ ]:
spend.info()

**Delinquency Variables**

- 96 total columns

- D_63: Object

- D_64: Object

- All others: float

- D_39, D_47, D_51, D_58, D_60, D_63, D_65, D_71, D_75, D_86, D_92, D_93, D_94, D_96, D_127 : no missing values

- D_42, D_49, D_66, D_73, D_76, D_87, D_88, D_106, D_108, D_110, D_111, D_132, D_134:D_138, D_142 : missing > 75% of values.

- D_41, D_44:D_46, D_48, D_52, D_54:D_55, D_59, D_61, D_62, D_64, D_68:D_70, D_72, D_74, D_78:D_81, D_83, D_84, D_89, D_91, D_102:D_104, D_107, D_109, D_112:D_126, D_128:D_131, D_133, D_139:D_145: missing < 25%

- D_43, D_50, D_53 D_56, D_77, D_82, D_105 : 25-75% missing



In [ ]:
delinq.D_63.value_counts()

In [ ]:
delinq.D_64.value_counts()

In [ ]:
delinq.info()

**Payment Variables**

- 3 total columns (P_2, P_3, P_4)

- all: float

- P_4 : no missing values

- P_2 & P_3 : missing < 1%

In [ ]:
pay.info()

In [ ]:
pay.describe()

**Balance Variables**

- 40 variables

- B_31: int (0, 1)

- all others: float

- B_29, B_39, and B_42 are majority null

- B_17 is missing 

- B_1, B_4, B_5, B_7, B_9, B_10, B_11, B_12, B_14, B_18, B_21, B_23, B_24, B_28, B_31, B_32, B_36 have no missing values. 

- B_2, B_3, B_6, B_8, B_13, B_15, B_16, B_19, B_20, B_25, B_26, B_27, B_30, B_33, B_37, B_38, B_40, B_41 are missing < 1% 


In [ ]:
balance.B_31.value_counts()

In [ ]:
balance.info()

In [ ]:
balance.describe().T

**Risk Variables**

- 28 Columns

- All: float

- R_9, R_26: missing > 90% of values. 

- R_12, R_20, and R_27 are missing < 1%

- R_1:R_8, R_10:R_11, R13:R19, R21:R26, R28 :  no missing values

In [ ]:
risk.info()

In [ ]:
# generate lists of column names by datatype for future use in analysis
object_cols = ['D_63', 'D_64']
int_cols = ['B_31']
date_cols = ['S_2']

# list of non_float columns in order to generate a list of all float column names (186 columns)
non_float_cols = object_cols + int_cols + date_cols
float_cols = [col for col in X_df.columns if col not in non_float_cols]
len(float_cols)

In [ ]:
with pd.option_context('display.max_rows', None,):
    print(null_df.sort_values('total_nulls'))

In [ ]:
null_df.groupby('feature_category').percent_nulls.agg(['mean', 'median', 'max', 'min']).sort_values('mean', ascending=False)

In [ ]:
y_df.target.value_counts(normalize=True)